# Solutions -- Tutorial 07: Complete Case Study (Monetary Policy Transmission)

**VAR Tutorial Series -- Exercise Solutions**

---

This notebook contains complete, worked solutions for all four exercises from Tutorial 07.
Each exercise includes:

1. The original problem statement
2. Step-by-step solution code
3. Interpretation and discussion

**Prerequisites**: You should have completed the main Tutorial 07 notebook before reviewing these solutions.

---

## Setup

In [ ]:
import sys
import os
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline
np.random.seed(42)
warnings.filterwarnings('ignore')

project_root = Path('../../../').resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

sys.path.insert(0, '../utils')

from panelbox.var import PanelVARData, PanelVAR

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams.update({'figure.figsize': (10, 6), 'figure.dpi': 100, 'font.size': 11})

print('Setup complete.')

## Load Data and Estimate Baseline Model

We first replicate the baseline 4-variable Panel VAR(2) estimated in the main tutorial.

In [ ]:
df = pd.read_csv('../data/monetary_policy.csv')
endog_vars = ['gdp_growth', 'inflation', 'interest_rate', 'unemployment']
data = PanelVARData(df, endog_vars=endog_vars, entity_col='country', time_col='quarter', lags=2)
model = PanelVAR(data)
results = model.fit(method='ols', cov_type='clustered')
print(f"Model estimated: {data.endog_names}")
print(f"Countries: {df['country'].nunique()}, Quarters: {df['quarter'].nunique()}")

In [ ]:
# Compute baseline IRFs for comparison in exercises
ordering_4var = ['interest_rate', 'inflation', 'gdp_growth', 'unemployment']
irf_4var = results.irf(
    periods=20,
    method='cholesky',
    order=ordering_4var,
    ci_method='bootstrap',
    n_bootstrap=500,
    ci_level=0.95,
    seed=42,
    verbose=False
)
print(f'Baseline IRF computed: {irf_4var.periods} periods, B=500')
print(f'CI available: {irf_4var.ci_lower is not None}')

---

## Exercise 1: Alternative Specifications (Easy)

**Task**: Re-estimate the baseline model with alternative specifications:
1. Only 3 variables (drop unemployment): `['interest_rate', 'inflation', 'gdp_growth']`
2. Exclude the crisis period (2008-2009) and compare IRFs
3. Compare the inflation response to a monetary shock across specifications

**Expected output**: IRF comparison plot, table of peak magnitudes, discussion of crisis-dependence

In [ ]:
# ============================================================
# Exercise 1 Solution: 3-variable VAR (drop unemployment)
# ============================================================

# Step 1: Define reduced variable set
endog_3var = ['gdp_growth', 'inflation', 'interest_rate']

# Step 2: Create PanelVARData and estimate
data_3var = PanelVARData(
    df,
    endog_vars=endog_3var,
    entity_col='country',
    time_col='quarter',
    lags=2
)
model_3var = PanelVAR(data_3var)
results_3var = model_3var.fit(method='ols', cov_type='clustered')

print('3-Variable Panel VAR(2) Estimation')
print('=' * 60)
print(f'  Variables: {endog_3var}')
print(f'  N = {results_3var.N}, n_obs = {results_3var.n_obs}')
print(f'  Stable: {results_3var.is_stable()}')
print(f'  Max eigenvalue modulus: {results_3var.max_eigenvalue_modulus:.6f}')

In [ ]:
# Step 3: Compute IRFs for the 3-variable model
ordering_3var = ['interest_rate', 'inflation', 'gdp_growth']

irf_3var = results_3var.irf(
    periods=20,
    method='cholesky',
    order=ordering_3var,
    ci_method='bootstrap',
    n_bootstrap=500,
    ci_level=0.95,
    seed=42,
    verbose=False
)

print(f'3-var IRF computed: {irf_3var.periods} periods, B=500')
print(f'Variables: {irf_3var.var_names}')

In [ ]:
# Step 4: Compare GDP response to interest rate shock
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
h = np.arange(21)

# --- Panel A: GDP growth response ---
ax = axes[0]

# 4-variable model
gdp_resp_4var = irf_4var['gdp_growth', 'interest_rate']
r4 = list(irf_4var.var_names).index('gdp_growth')
i4 = list(irf_4var.var_names).index('interest_rate')
if irf_4var.ci_lower is not None:
    ax.fill_between(h, irf_4var.ci_lower[:, r4, i4], irf_4var.ci_upper[:, r4, i4],
                    alpha=0.15, color='#2166ac')
ax.plot(h, gdp_resp_4var, color='#2166ac', linewidth=2.2,
        marker='o', markersize=3, label='4-var (with unemployment)')

# 3-variable model
gdp_resp_3var = irf_3var['gdp_growth', 'interest_rate']
r3 = list(irf_3var.var_names).index('gdp_growth')
i3 = list(irf_3var.var_names).index('interest_rate')
if irf_3var.ci_lower is not None:
    ax.fill_between(h, irf_3var.ci_lower[:, r3, i3], irf_3var.ci_upper[:, r3, i3],
                    alpha=0.15, color='#b2182b')
ax.plot(h, gdp_resp_3var, color='#b2182b', linewidth=2.2,
        marker='s', markersize=3, label='3-var (no unemployment)', linestyle='--')

ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
ax.set_title('GDP Growth Response to Monetary Shock', fontsize=13, fontweight='bold')
ax.set_xlabel('Horizon (quarters)', fontsize=11)
ax.set_ylabel('Response', fontsize=11)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# --- Panel B: Inflation response ---
ax = axes[1]

inf_resp_4var = irf_4var['inflation', 'interest_rate']
ri4 = list(irf_4var.var_names).index('inflation')
ax.plot(h, inf_resp_4var, color='#2166ac', linewidth=2.2,
        marker='o', markersize=3, label='4-var (with unemployment)')

inf_resp_3var = irf_3var['inflation', 'interest_rate']
ri3 = list(irf_3var.var_names).index('inflation')
ax.plot(h, inf_resp_3var, color='#b2182b', linewidth=2.2,
        marker='s', markersize=3, label='3-var (no unemployment)', linestyle='--')

ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
ax.set_title('Inflation Response to Monetary Shock', fontsize=13, fontweight='bold')
ax.set_xlabel('Horizon (quarters)', fontsize=11)
ax.set_ylabel('Response', fontsize=11)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

fig.suptitle('Exercise 1: Comparing 3-var vs 4-var Specifications',
             fontsize=14, fontweight='bold', y=1.02)
fig.tight_layout()
plt.show()

In [ ]:
# Step 5: Quantitative comparison table
print('Comparison: GDP Growth Response to Monetary Shock')
print('=' * 75)
print(f'{"Horizon":>8s} {"4-var model":>15s} {"3-var model":>15s} {"Difference":>15s}')
print('-' * 75)

for horizon in [0, 1, 2, 4, 8, 12, 16, 20]:
    val_4 = gdp_resp_4var[horizon]
    val_3 = gdp_resp_3var[horizon]
    diff = val_3 - val_4
    print(f'{horizon:>8d} {val_4:>15.6f} {val_3:>15.6f} {diff:>15.6f}')

# Peak effects
peak_4 = np.argmax(np.abs(gdp_resp_4var))
peak_3 = np.argmax(np.abs(gdp_resp_3var))

print(f'\nPeak Effects:')
print(f'  4-var: {gdp_resp_4var[peak_4]:.6f} at h={peak_4}')
print(f'  3-var: {gdp_resp_3var[peak_3]:.6f} at h={peak_3}')

print('\nInterpretation:')
print('  Dropping unemployment alters the IRF somewhat because unemployment')
print('  absorbs part of the labor market adjustment channel. Without it,')
print('  the GDP response may appear somewhat different in magnitude or timing.')
print('  However, the qualitative direction should remain consistent:')
print('  a contractionary monetary shock reduces GDP growth.')

In [ ]:
# ============================================================
# Step 6: Crisis exclusion analysis (2008-2009)
# ============================================================

# Exclude crisis quarters (2008-Q1 through 2009-Q4)
crisis_quarters = [q for q in df['quarter'].unique()
                   if q.startswith('2008') or q.startswith('2009')]
print(f'Crisis quarters excluded: {crisis_quarters}')

df_no_crisis = df[~df['quarter'].isin(crisis_quarters)].copy()
print(f'Full sample: {len(df)} rows, No-crisis sample: {len(df_no_crisis)} rows')
print(f'Quarters removed: {len(crisis_quarters)}')

# Estimate VAR on no-crisis sample
data_nc = PanelVARData(
    df_no_crisis, endog_vars=endog_vars,
    entity_col='country', time_col='quarter', lags=2
)
model_nc = PanelVAR(data_nc)
results_nc = model_nc.fit(method='ols', cov_type='driscoll_kraay')

irf_nc = results_nc.irf(
    periods=20, method='cholesky', order=ordering_4var,
    ci_method='bootstrap', n_bootstrap=500, ci_level=0.95,
    seed=42, verbose=False
)

print(f'\nNo-crisis VAR(2): N={results_nc.N}, n_obs={results_nc.n_obs}')
print(f'Stable: {results_nc.is_stable()}')

In [ ]:
# Step 7: Compare IRFs: Full sample vs No-crisis vs 3-var
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
h = np.arange(21)

# Panel A: Inflation response comparison
ax = axes[0]
ax.plot(h, irf_4var['inflation', 'interest_rate'], color='#2166ac',
        linewidth=2.2, label='Full sample (4-var)', marker='o', markersize=3)
ax.plot(h, irf_nc['inflation', 'interest_rate'], color='#b2182b',
        linewidth=2.2, label='Excl. crisis (4-var)', marker='s', markersize=3, linestyle='--')
ax.plot(h, irf_3var['inflation', 'interest_rate'], color='#4dac26',
        linewidth=2.2, label='Full sample (3-var)', marker='^', markersize=3, linestyle='-.')
ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
ax.set_title('Inflation Response to Monetary Shock', fontsize=13, fontweight='bold')
ax.set_xlabel('Horizon (quarters)', fontsize=11)
ax.set_ylabel('Response', fontsize=11)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

# Panel B: GDP growth response comparison
ax = axes[1]
ax.plot(h, irf_4var['gdp_growth', 'interest_rate'], color='#2166ac',
        linewidth=2.2, label='Full sample (4-var)', marker='o', markersize=3)
ax.plot(h, irf_nc['gdp_growth', 'interest_rate'], color='#b2182b',
        linewidth=2.2, label='Excl. crisis (4-var)', marker='s', markersize=3, linestyle='--')
ax.plot(h, irf_3var['gdp_growth', 'interest_rate'], color='#4dac26',
        linewidth=2.2, label='Full sample (3-var)', marker='^', markersize=3, linestyle='-.')
ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
ax.set_title('GDP Growth Response to Monetary Shock', fontsize=13, fontweight='bold')
ax.set_xlabel('Horizon (quarters)', fontsize=11)
ax.set_ylabel('Response', fontsize=11)
ax.legend(fontsize=9)
ax.grid(True, alpha=0.3)

fig.suptitle('Exercise 1: Robustness across Specifications\n'
             '(Full sample vs Crisis-excluded vs Reduced variable set)',
             fontsize=14, fontweight='bold', y=1.03)
fig.tight_layout()
plt.show()

In [ ]:
# Step 8: Summary table of peak IRF magnitudes across specifications
print('Peak IRF Magnitudes Across Specifications')
print('=' * 95)
print(f'{"Specification":>25s} {"Variable":>16s} {"Peak Value":>12s} {"Peak Horizon":>14s}')
print('-' * 95)

specs = {
    'Full sample (4-var)': irf_4var,
    'Excl. crisis (4-var)': irf_nc,
    'Full sample (3-var)': irf_3var,
}

for spec_name, irf_obj in specs.items():
    for resp_var in ['inflation', 'gdp_growth']:
        if resp_var in irf_obj.var_names and 'interest_rate' in irf_obj.var_names:
            vals = irf_obj[resp_var, 'interest_rate']
            peak_h = np.argmax(np.abs(vals))
            print(f'{spec_name:>25s} {resp_var:>16s} {vals[peak_h]:>12.6f} {peak_h:>14d}')

print('\nDiscussion:')
print('  The crisis period (2008-2009) was characterized by extraordinary monetary')
print('  policy actions (near-zero rates, quantitative easing). Excluding this period')
print('  tests whether the baseline results are driven by crisis dynamics or reflect')
print('  normal-times monetary transmission. Similar IRF patterns across specifications')
print('  support the robustness of our findings.')

---

## Exercise 2: Country-by-Country Heterogeneity (Medium)

**Task**: Estimate individual VARs for each country (USA, DEU, JPN, MEX, KOR). Compare the inflation IRF to a monetary policy shock across countries. Calculate half-lives and create a bar chart.

**Expected output:**
- Multi-panel figure showing country-specific inflation IRFs overlaid on the panel IRF
- Table of peak response magnitudes and timing by country
- Bar chart of half-life of inflation adjustment by country

In [ ]:
# ============================================================
# Exercise 2 Solution: Country-by-country estimation
# ============================================================

countries_to_analyze = ['USA', 'DEU', 'JPN', 'MEX', 'KOR']
country_colors = {
    'USA': '#2166ac',
    'DEU': '#b2182b',
    'JPN': '#4dac26',
    'MEX': '#7b3294',
    'KOR': '#e08214',
}
country_styles = {
    'USA': '-',
    'DEU': '--',
    'JPN': '-.',
    'MEX': ':',
    'KOR': '-',
}

# Step 1: Estimate separate VARs for each country
country_results = {}
country_irfs = {}

ordering_country = ['interest_rate', 'inflation', 'gdp_growth', 'unemployment']

print('Country-by-Country VAR(2) Estimation')
print('=' * 70)

for country in countries_to_analyze:
    df_c = df[df['country'] == country].copy()
    
    try:
        data_c = PanelVARData(
            df_c,
            endog_vars=endog_vars,
            entity_col='country',
            time_col='quarter',
            lags=2
        )
        model_c = PanelVAR(data_c)
        results_c = model_c.fit(method='ols', cov_type='nonrobust')
        
        irf_c = results_c.irf(
            periods=20,
            method='cholesky',
            order=ordering_country
        )
        
        country_results[country] = results_c
        country_irfs[country] = irf_c
        
        print(f'  {country}: N={results_c.N}, T_eff={results_c.n_obs}, '
              f'stable={results_c.is_stable()}, '
              f'max_eig={results_c.max_eigenvalue_modulus:.4f}')
    except Exception as e:
        print(f'  {country}: FAILED -- {e}')

print(f'\nSuccessfully estimated: {len(country_results)} countries')

In [ ]:
# Step 2: Plot all country-specific GDP responses on one figure
fig, axes = plt.subplots(1, 3, figsize=(20, 6))
h = np.arange(21)

response_vars_plot = ['gdp_growth', 'inflation', 'unemployment']
titles_plot = {
    'gdp_growth': 'GDP Growth',
    'inflation': 'Inflation',
    'unemployment': 'Unemployment'
}

for idx, resp_var in enumerate(response_vars_plot):
    ax = axes[idx]
    
    for country in countries_to_analyze:
        if country in country_irfs:
            irf_c = country_irfs[country]
            vals = irf_c[resp_var, 'interest_rate']
            ax.plot(h, vals,
                    color=country_colors[country],
                    linewidth=2,
                    linestyle=country_styles[country],
                    label=country)
    
    ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
    ax.set_title(f'{titles_plot[resp_var]} Response', fontsize=13, fontweight='bold')
    ax.set_xlabel('Horizon (quarters)', fontsize=11)
    ax.set_ylabel('Response', fontsize=11)
    ax.legend(fontsize=9, loc='best')
    ax.grid(True, alpha=0.3)

fig.suptitle('Country-Specific Responses to Monetary Policy Shock\n'
             '(Individual Time-Series VARs, Cholesky Identification)',
             fontsize=14, fontweight='bold', y=1.03)
fig.tight_layout()
plt.show()

In [ ]:
# Step 2b: Inflation IRF comparison with panel IRF overlay
fig, ax = plt.subplots(figsize=(12, 7))
h = np.arange(21)

# Panel IRF (thick black)
panel_inf = irf_4var['inflation', 'interest_rate']
r_inf = list(irf_4var.var_names).index('inflation')
i_ir = list(irf_4var.var_names).index('interest_rate')
if irf_4var.ci_lower is not None:
    ax.fill_between(h, irf_4var.ci_lower[:, r_inf, i_ir],
                    irf_4var.ci_upper[:, r_inf, i_ir],
                    alpha=0.12, color='gray')
ax.plot(h, panel_inf, color='black', linewidth=3, label='Panel IRF', zorder=10)

# Country-specific IRFs
for country in countries_to_analyze:
    if country in country_irfs:
        vals = country_irfs[country]['inflation', 'interest_rate']
        ax.plot(h, vals, color=country_colors[country], linewidth=1.8,
                linestyle=country_styles[country], label=country, alpha=0.8)

ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
ax.set_title('Inflation Response to Monetary Shock:\nCountry-Specific vs Panel IRF',
             fontsize=14, fontweight='bold')
ax.set_xlabel('Horizon (quarters)', fontsize=12)
ax.set_ylabel('Response', fontsize=12)
ax.legend(fontsize=10, loc='best')
ax.grid(True, alpha=0.3)
fig.tight_layout()
plt.show()

In [ ]:
# Step 3: Create comparison table of peak effects
print('Peak GDP Growth Response to Monetary Shock by Country')
print('=' * 80)
print(f'{"Country":>8s} {"Peak Value":>12s} {"Peak Horizon":>14s} '
      f'{"Impact (h=0)":>14s} {"Long-run (h=20)":>16s}')
print('-' * 80)

peak_effects = []

for country in countries_to_analyze:
    if country in country_irfs:
        vals = country_irfs[country]['gdp_growth', 'interest_rate']
        peak_h = np.argmax(np.abs(vals))
        peak_val = vals[peak_h]
        peak_effects.append({
            'Country': country,
            'Peak_Value': peak_val,
            'Peak_Horizon': peak_h,
            'Impact': vals[0],
            'Long_Run': vals[-1]
        })
        print(f'{country:>8s} {peak_val:>12.6f} {peak_h:>14d} '
              f'{vals[0]:>14.6f} {vals[-1]:>16.6f}')

# Identify strongest and weakest
peak_df = pd.DataFrame(peak_effects)
strongest = peak_df.loc[peak_df['Peak_Value'].abs().idxmax(), 'Country']
weakest = peak_df.loc[peak_df['Peak_Value'].abs().idxmin(), 'Country']

print(f'\nStrongest monetary transmission: {strongest}')
print(f'Weakest monetary transmission:   {weakest}')

print('\nInterpretation:')
print('  Country-level heterogeneity reveals that monetary policy transmission')
print('  varies considerably across economies. This reflects differences in:')
print('  - Financial market depth and the interest rate channel')
print('  - Exchange rate regime and openness')
print('  - Central bank credibility and inflation expectations anchoring')
print('  - Structural labor market rigidities')

In [ ]:
# Step 4: Compute half-life of inflation adjustment for each country
# Half-life = number of periods after peak until |response| <= |peak|/2

def compute_half_life(irf_vals):
    """Compute half-life: periods after peak until response decays to half peak value."""
    peak_h = np.argmax(np.abs(irf_vals))
    peak_val = irf_vals[peak_h]
    if abs(peak_val) < 1e-10:
        return 0
    half_target = abs(peak_val) / 2.0
    for hh in range(peak_h, len(irf_vals)):
        if abs(irf_vals[hh]) <= half_target:
            return hh - peak_h
    return len(irf_vals) - 1 - peak_h  # didn't decay to half

# Also include the panel IRF for comparison
panel_inf_vals = irf_4var['inflation', 'interest_rate']
panel_half_life = compute_half_life(panel_inf_vals)

half_life_data = []
print('Half-Life of Inflation Adjustment by Country')
print('=' * 65)
print(f'{"Country":>8s} {"Peak Effect":>14s} {"Peak Horizon":>14s} {"Half-Life":>12s}')
print('-' * 65)

for country in countries_to_analyze:
    if country in country_irfs:
        inf_vals = country_irfs[country]['inflation', 'interest_rate']
        peak_h = np.argmax(np.abs(inf_vals))
        peak_val = inf_vals[peak_h]
        hl = compute_half_life(inf_vals)
        half_life_data.append({'Country': country, 'Peak': peak_val,
                               'Peak_Horizon': peak_h, 'Half_Life': hl})
        print(f'{country:>8s} {peak_val:>14.6f} {peak_h:>14d} {hl:>12d}')

print(f'{"Panel":>8s} {panel_inf_vals[np.argmax(np.abs(panel_inf_vals))]:>14.6f} '
      f'{np.argmax(np.abs(panel_inf_vals)):>14d} {panel_half_life:>12d}')

# Bar chart of half-lives
hl_df = pd.DataFrame(half_life_data)

fig, ax = plt.subplots(figsize=(10, 6))
colors_hl = [country_colors.get(c, '#666') for c in hl_df['Country']]
bars = ax.bar(hl_df['Country'], hl_df['Half_Life'], color=colors_hl, edgecolor='white',
              linewidth=1.5, alpha=0.85)

# Add panel average line
ax.axhline(y=panel_half_life, color='black', linewidth=2, linestyle='--',
           label=f'Panel average ({panel_half_life} quarters)')

# Add value labels on bars
for bar, val in zip(bars, hl_df['Half_Life']):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.2,
            f'{val}Q', ha='center', va='bottom', fontsize=11, fontweight='bold')

ax.set_xlabel('Country', fontsize=12)
ax.set_ylabel('Half-Life (quarters)', fontsize=12)
ax.set_title('Half-Life of Inflation Adjustment to Monetary Shock\n'
             '(Quarters until response decays to half peak value)',
             fontsize=14, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y')
fig.tight_layout()
plt.show()

print('\nCountries with shorter half-lives have more responsive inflation dynamics.')
print('This may reflect more flexible pricing, better-anchored expectations, or')
print('stronger monetary policy credibility.')

---

## Exercise 3: GMM Extension for Short Panels (Hard)

**Task**: Subset the data to T=15 quarters (simulating a short panel). Compare OLS estimates from the full sample (T=80) vs the short panel (T=15). Analyze Nickell bias and discuss when GMM would be necessary.

**Expected output:**
- Side-by-side comparison table: Full sample vs Short panel coefficient estimates
- Diagnostic summary (stability, information criteria)
- IRF comparison plot across estimation samples
- Theoretical Nickell bias chart as function of T

In [ ]:
# ============================================================
# Exercise 3 Solution: Short Panel (T=15) vs Full Panel (T=80)
# ============================================================

# Step 1: Full sample baseline (already computed)
print('Step 1: Full Sample OLS Baseline')
print('=' * 60)
print(f'  Method: OLS with fixed effects')
print(f'  Covariance: clustered')
print(f'  N = {results.N}, K = {results.K}, p = {results.p}')
print(f'  T = {df["quarter"].nunique()} quarters')
print(f'  n_obs = {results.n_obs}')
print(f'  Stable: {results.is_stable()}')
print(f'  Max eigenvalue: {results.max_eigenvalue_modulus:.6f}')
print(f'  AIC: {results.aic:.4f}')
print(f'  BIC: {results.bic:.4f}')

In [ ]:
# Step 2: Create short panel (T=15 most recent quarters)
quarters_sorted = sorted(df['quarter'].unique())
last_15 = quarters_sorted[-15:]
print(f'Step 2: Short Panel (T=15)')
print('=' * 60)
print(f'  Selected quarters: {last_15[0]} to {last_15[-1]}')

df_short = df[df['quarter'].isin(last_15)].copy()
print(f'  Full sample: N={df["country"].nunique()}, T={len(quarters_sorted)}, '
      f'rows={len(df)}')
print(f'  Short panel: N={df_short["country"].nunique()}, T={len(last_15)}, '
      f'rows={len(df_short)}')

# Estimate VAR on short panel
data_short = PanelVARData(
    df_short, endog_vars=endog_vars,
    entity_col='country', time_col='quarter', lags=2
)
model_short = PanelVAR(data_short)
results_short = model_short.fit(method='ols', cov_type='clustered')

print(f'\n  Short Panel VAR(2):')
print(f'    n_obs = {results_short.n_obs}')
print(f'    Stable: {results_short.is_stable()}')
print(f'    Max eigenvalue: {results_short.max_eigenvalue_modulus:.6f}')
print(f'    AIC: {results_short.aic:.4f}')
print(f'    BIC: {results_short.bic:.4f}')

In [ ]:
# Step 3: Compare coefficient matrices (Lag 1)
print('Step 3: Coefficient Comparison (A_1 matrix)')
print('=' * 80)

A1_full = results.coef_matrix(lag=1)
A1_short = results_short.coef_matrix(lag=1)

print('\nA_1 (Full Sample, T=80):')
print(A1_full.round(4).to_string())

print('\nA_1 (Short Panel, T=15):')
print(A1_short.round(4).to_string())

print('\nDifference (Short - Full):')
diff = A1_short - A1_full
print(diff.round(4).to_string())

print('\nMean absolute difference:', diff.abs().values.mean().round(4))
print('Max absolute difference: ', diff.abs().values.max().round(4))

# Theoretical Nickell bias
N = df['country'].nunique()
T_full = len(quarters_sorted)
T_short = 15

print(f'\nNickell Bias Analysis:')
print(f'  Full sample (T={T_full}):')
rho = 0.5  # assumed
bias_full = -(1 + rho) / (T_full - 1)
bias_short = -(1 + rho) / (T_short - 1)
print(f'    Bias ~ {bias_full:.6f} ({abs(bias_full/rho)*100:.1f}% of rho={rho})')
print(f'  Short panel (T={T_short}):')
print(f'    Bias ~ {bias_short:.6f} ({abs(bias_short/rho)*100:.1f}% of rho={rho})')
print(f'  Ratio: Short panel bias is {abs(bias_short/bias_full):.1f}x larger')

In [ ]:
# Step 4: IRF comparison and Nickell bias visualization
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
h = np.arange(21)

# Panel A: IRF comparison (full vs short panel)
ax = axes[0]
irf_short = results_short.irf(periods=20, method='cholesky', order=ordering_4var)

for resp_var, color, marker in [('gdp_growth', '#2166ac', 'o'),
                                  ('inflation', '#b2182b', 's')]:
    vals_full = irf_4var[resp_var, 'interest_rate']
    vals_short = irf_short[resp_var, 'interest_rate']
    label_base = resp_var.replace('_', ' ').title()
    ax.plot(h, vals_full, color=color, linewidth=2, marker=marker, markersize=3,
            label=f'{label_base} (T=80)')
    ax.plot(h, vals_short, color=color, linewidth=2, marker=marker, markersize=3,
            label=f'{label_base} (T=15)', linestyle='--', alpha=0.7)

ax.axhline(y=0, color='black', linewidth=0.8, linestyle='--', alpha=0.5)
ax.set_title('IRF Comparison: Full vs Short Panel', fontsize=13, fontweight='bold')
ax.set_xlabel('Horizon (quarters)', fontsize=11)
ax.set_ylabel('Response', fontsize=11)
ax.legend(fontsize=8, loc='best')
ax.grid(True, alpha=0.3)

# Panel B: Nickell bias as function of T
ax = axes[1]
T_range = np.arange(5, 101)
rho_values = [0.3, 0.5, 0.7, 0.9]
colors_bias = ['#2166ac', '#4dac26', '#d6604d', '#7b3294']

for rho_val, color in zip(rho_values, colors_bias):
    bias_pct = np.abs(-(1 + rho_val) / (T_range - 1) / rho_val) * 100
    ax.plot(T_range, bias_pct, color=color, linewidth=2, label=f'rho = {rho_val}')

ax.axvline(x=T_full, color='green', linewidth=1.5, linestyle='--', alpha=0.7,
           label=f'Full sample (T={T_full})')
ax.axvline(x=T_short, color='red', linewidth=1.5, linestyle='--', alpha=0.7,
           label=f'Short panel (T={T_short})')
ax.axhline(y=5, color='gray', linewidth=0.8, linestyle=':', alpha=0.5)
ax.text(T_short + 1, 70, f'T={T_short}', fontsize=10, color='red')
ax.text(T_full + 1, 50, f'T={T_full}', fontsize=10, color='green')

ax.set_xlabel('Time periods (T)', fontsize=12)
ax.set_ylabel('Bias as % of true parameter', fontsize=12)
ax.set_title('Nickell Bias Magnitude', fontsize=13, fontweight='bold')
ax.legend(fontsize=8, loc='upper right')
ax.set_xlim(5, 100)
ax.set_ylim(0, 80)
ax.grid(True, alpha=0.3)

fig.suptitle('Exercise 3: Short Panel Analysis and Nickell Bias',
             fontsize=14, fontweight='bold', y=1.02)
fig.tight_layout()
plt.show()

print(f'At T={T_short}, Nickell bias is ~{abs(bias_short/rho)*100:.0f}% -- GMM correction recommended.')
print(f'At T={T_full}, Nickell bias is ~{abs(bias_full/rho)*100:.0f}% -- OLS is reliable.')

### Discussion: OLS vs GMM for Panel VARs

**Key Points:**

1. **Nickell bias** arises because the within-transformation creates a correlation between the demeaned lagged dependent variable and the demeaned error term. The bias is of order O(1/T).

2. **Full panel (N=25, T=80)**: The bias is approximately 1.9% of the true coefficient value (for rho=0.5). This is negligible and well within sampling variability.

3. **Short panel (N=25, T=15)**: The bias increases to approximately 10.7% -- now potentially meaningful. At T=15, GMM correction (Arellano-Bond or System GMM) would be recommended.

4. **Coefficient comparison**: The differences between full-sample and short-panel estimates reflect both Nickell bias AND the different sample periods. With only T=15 post-crisis quarters, structural differences from the pre-crisis period are not captured.

5. **Practical rule of thumb**: 
   - T > 30: OLS with fixed effects is reliable; Nickell bias < 5%
   - T = 15-30: OLS may be acceptable but GMM provides robustness check
   - T < 15: GMM correction is strongly recommended

6. **GMM challenges**: Arellano-Bond GMM introduces its own complications -- instrument proliferation, weak instruments, and finite-sample bias from too many moment conditions -- which can be more problematic than the Nickell bias it corrects. System GMM is preferred but requires additional assumptions (stationarity of initial conditions).

---

## Exercise 4: Policy Brief (Medium)

**Task**: Write a structured 2-page policy brief summarizing the main findings. Include: (a) key stylized facts from EDA, (b) main IRF results with confidence intervals, (c) FEVD decomposition at policy-relevant horizons, (d) heterogeneity insights, and (e) policy recommendations.

**Expected output**: Markdown cell with formatted policy brief, at least 2 publication-quality figures, executive summary, methodology, key findings, and policy implications.

In [ ]:
# ============================================================
# Exercise 4 Solution: Policy Brief
# ============================================================

# Step 1: Compute key statistics for the brief

# GDP response to monetary shock
gdp_resp = irf_4var['gdp_growth', 'interest_rate']
gdp_peak_h = np.argmax(np.abs(gdp_resp))
gdp_peak_val = gdp_resp[gdp_peak_h]

# Half-life: when does the response decay to half its peak?
half_target = gdp_peak_val / 2.0
half_life = gdp_peak_h
for hh in range(gdp_peak_h, len(gdp_resp)):
    if abs(gdp_resp[hh]) <= abs(half_target):
        half_life = hh
        break

# Significance horizon: how many periods is the response outside the CI of zero?
# (i.e., how many periods is the CI entirely below or above zero)
r_idx = list(irf_4var.var_names).index('gdp_growth')
i_idx = list(irf_4var.var_names).index('interest_rate')
sig_horizon = 0
if irf_4var.ci_lower is not None:
    for hh in range(len(gdp_resp)):
        ci_lo = irf_4var.ci_lower[hh, r_idx, i_idx]
        ci_hi = irf_4var.ci_upper[hh, r_idx, i_idx]
        # Significant if both bounds have the same sign (CI excludes zero)
        if (ci_lo > 0 and ci_hi > 0) or (ci_lo < 0 and ci_hi < 0):
            sig_horizon = hh  # keep updating to find the last significant horizon

print('Key Statistics for Policy Brief')
print('=' * 50)
print(f'GDP Peak response:      {gdp_peak_val:.4f}')
print(f'GDP Peak horizon:       h={gdp_peak_h} quarters')
print(f'GDP Half-life:          ~h={half_life} quarters')
print(f'Significance horizon:   h={sig_horizon} quarters')
print(f'GDP Long-run (h=20):    {gdp_resp[-1]:.6f}')

In [ ]:
# Step 2: Create publication-quality figure
fig, ax = plt.subplots(figsize=(10, 6))
h = np.arange(irf_4var.periods + 1)

# Plot confidence interval
if irf_4var.ci_lower is not None:
    ci_lo = irf_4var.ci_lower[:, r_idx, i_idx]
    ci_hi = irf_4var.ci_upper[:, r_idx, i_idx]
    ax.fill_between(h, ci_lo, ci_hi, alpha=0.20, color='#2166ac',
                    label='95% confidence interval')

# Plot point estimate
ax.plot(h, gdp_resp, color='#2166ac', linewidth=2.5, marker='o',
        markersize=4, label='Point estimate', zorder=5)

# Zero line
ax.axhline(y=0, color='black', linewidth=1.0, linestyle='-', alpha=0.4)

# Annotate peak
ax.annotate(
    f'Peak: {gdp_peak_val:.4f}\nat quarter {gdp_peak_h}',
    xy=(gdp_peak_h, gdp_peak_val),
    xytext=(gdp_peak_h + 3, gdp_peak_val - 0.02 * np.sign(gdp_peak_val)),
    fontsize=10,
    arrowprops=dict(arrowstyle='->', color='#333333', lw=1.5),
    bbox=dict(boxstyle='round,pad=0.4', facecolor='lightyellow',
              edgecolor='#999999', alpha=0.9)
)

ax.set_xlabel('Quarters after shock', fontsize=13)
ax.set_ylabel('Response of GDP growth (pp)', fontsize=13)
ax.set_title('How Does GDP Growth Respond to a Monetary Policy Tightening?',
             fontsize=14, fontweight='bold', pad=15)
ax.legend(fontsize=11, loc='lower right')
ax.grid(True, alpha=0.3)
ax.set_xlim(-0.5, 20.5)

fig.tight_layout()
fig.savefig('../outputs/figures/irfs/07_policy_brief_figure.png',
            dpi=300, bbox_inches='tight')
plt.show()

print('Figure saved: ../outputs/figures/irfs/07_policy_brief_figure.png')

In [ ]:
# Step 2b: Second publication-quality figure - FEVD at policy-relevant horizons
# Re-compute FEVD using same ordering as main notebook
from panelbox.var import PanelVARData, PanelVAR

data_brief = PanelVARData(df, endog_vars=endog_vars, entity_col='country',
                          time_col='quarter', lags=2)
results_brief = PanelVAR(data_brief).fit(method='ols', cov_type='driscoll_kraay')
fevd_brief = results_brief.fevd(periods=20, method='cholesky', order=ordering_4var)

# Create a grouped bar chart of FEVD at key horizons for GDP growth
fig, ax = plt.subplots(figsize=(10, 6))

horizons_plot = [1, 4, 8, 20]
gdp_idx = list(fevd_brief.var_names).index('gdp_growth')
n_shocks = fevd_brief.K
x = np.arange(len(horizons_plot))
bar_width = 0.18
colors_fevd = ['#2166ac', '#d6604d', '#4dac26', '#7b3294']

for j in range(n_shocks):
    shares = [fevd_brief.decomposition[h, gdp_idx, j] * 100 for h in horizons_plot]
    offset = (j - n_shocks / 2 + 0.5) * bar_width
    bars = ax.bar(x + offset, shares, bar_width, label=fevd_brief.var_names[j],
                  color=colors_fevd[j], alpha=0.85, edgecolor='white')

ax.set_xlabel('Forecast Horizon (quarters)', fontsize=13)
ax.set_ylabel('Variance Share (%)', fontsize=13)
ax.set_title('What Drives GDP Growth Variability?\nForecast Error Variance Decomposition',
             fontsize=14, fontweight='bold', pad=15)
ax.set_xticks(x)
ax.set_xticklabels([f'h={h}' for h in horizons_plot], fontsize=11)
ax.legend(fontsize=10, title='Shock source', title_fontsize=11)
ax.set_ylim(0, 105)
ax.grid(True, alpha=0.3, axis='y')

fig.tight_layout()
fig.savefig('../outputs/figures/fevds/07_policy_brief_fevd.png',
            dpi=300, bbox_inches='tight')
plt.show()
print('Figure saved: ../outputs/figures/fevds/07_policy_brief_fevd.png')

In [ ]:
# Step 3: Print the formatted policy brief
print('=' * 72)
print('              POLICY BRIEF: MONETARY POLICY TRANSMISSION')
print('              IN A CROSS-COUNTRY PANEL (2000-2019)')
print('=' * 72)

print('''
KEY QUESTION: Does monetary policy work?

SHORT ANSWER: Yes, but with important caveats about timing and
heterogeneity across countries.

METHODOLOGY
-----------
We estimate a Panel Vector Autoregression (VAR) using quarterly
data from 25 countries over 2000-2019 (2,000 country-quarter
observations). The model includes four macroeconomic variables:
GDP growth, inflation, the policy interest rate, and unemployment.
Structural shocks are identified using a Cholesky decomposition
with standard macroeconomic ordering.
''')

print('KEY FINDINGS')
print('-' * 40)
print(f'''
  1. MONETARY POLICY IS EFFECTIVE:
     A one-standard-deviation contractionary monetary shock
     (interest rate increase) reduces GDP growth by up to
     {abs(gdp_peak_val):.3f} percentage points.

  2. TRANSMISSION IS GRADUAL:
     The peak effect on GDP occurs at quarter {gdp_peak_h},
     confirming Friedman\'s "long and variable lags." The
     response half-life is approximately {half_life} quarters.

  3. HETEROGENEITY MATTERS:
     Advanced economies (USA, Germany, Japan) show different
     transmission patterns compared to emerging/transition
     economies (Mexico, Hungary, Poland, Czech Republic, Korea).
     This reflects differences in financial market depth,
     central bank credibility, and structural rigidities.

  4. ROBUSTNESS:
     Results are qualitatively robust to alternative variable
     orderings, lag specifications, and country groupings.
''')

print('POLICY IMPLICATIONS')
print('-' * 40)
print('''
  - Central banks should anticipate delayed effects when
    adjusting interest rates. The full impact on GDP takes
    several quarters to materialize.

  - "One size fits all" monetary policy rules are inappropriate
    given the substantial cross-country heterogeneity in
    transmission mechanisms.

  - Communication and forward guidance are crucial, as the
    expectations channel amplifies or dampens the direct
    interest rate channel depending on central bank credibility.
''')

print('=' * 72)
print('  Data: 25 countries, quarterly, 2000-2019')
print('  Method: Panel VAR(2) with Cholesky identification')
print('  Software: PanelBox (Python)')
print('=' * 72)

---

## Summary

| Exercise | Topic | Difficulty | Key Takeaway |
|----------|-------|-----------|--------------|
| 1 | Alternative Specifications | Easy | Dropping unemployment or excluding crisis period changes magnitudes slightly but not qualitative conclusions |
| 2 | Country Heterogeneity | Medium | Individual country VARs reveal substantial differences in transmission speed and magnitude; half-life varies by country |
| 3 | Short Panel & Nickell Bias | Hard | With T=15, Nickell bias is ~11% (GMM recommended); with T=80, bias is ~2% (OLS reliable) |
| 4 | Policy Brief | Medium | Monetary policy works but with long lags and cross-country heterogeneity; forward guidance essential |

---

**End of Solutions for Tutorial 07**